In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt

from compute_tempo import *
from extract_dance_onsets import *
# from aist_pos1s_EsTempo import *
# coco={    
# 0: "nose", 1: "left_eye", 2: "right_eye", 3: "left_ear",4: "right_ear",5: "left_shoulder",
# 6: "right_shoulder",7: "left_elbow",8: "right_elbow",9: "left_wrist",10: "right_wrist",
# 11: "left_hip",12: "right_hip",13: "left_knee",14: "right_knee",15: "left_ankle",16: "right_ankle",}  

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        json_data = pickle.load(f)
    return json_data

def save_to_pickle(filepath, data):
    # filepath = os.path.join(savepath, filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f)
        
def create_onset_dir(main_dir, tempo_dir):
    # main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
    directories = [f"{tempo_dir}/pos", f"{tempo_dir}/pos/ax0",
                   f"{tempo_dir}/pos/ax1", f"{tempo_dir}/pos/combination",
                   f"{tempo_dir}/pos/resultant",
                   
                   f"{tempo_dir}/vel", f"{tempo_dir}/vel/ax0",
                   f"{tempo_dir}/vel/ax1", f"{tempo_dir}/vel/combination",
                   f"{tempo_dir}/vel/resultant",]
    
    for dir_path in directories:
        full_path = os.path.join(main_dir, dir_path)
        os.makedirs(full_path, exist_ok=True)
        
# create_onset_dir("/itf-fi-ml/home/sagardu/aist_tempo_est/extracted_body_onsets", "thres_0.4")

## March 11 Updates

## Onset Extraction Cell

In [ ]:
# marker_dict = {9: "left_wrist", 10: "right_wrist", 
#                 15: "left_ankle", 16: "right_ankle", 
#                 }   # 11: "left_hip",12: "right_hip"

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": [9, 10], "a": 60, "b": 140, "metric": ["pos", "vel"]}
config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"],
           "markerA_id": [15, 16], "a": 60, "b": 140, "metric": ["pos", "vel"]}

configs = [config1, config2]
# create_onset_dir("/itf-fi-ml/home/sagardu/aist_tempo_est/extracted_body_onsets", "thres_0.4")
for cfg in configs:
    a = cfg["a"]
    b = cfg["b"]
    
    for sub_dir in cfg["sub_dir"]:
        for mode in cfg["mode"]:
            for markerA_id in cfg["markerA_id"]:
                for metric in cfg["metric"]:
                    
                    savepath = f"./extracted_body_onsets/{metric}"           
                    extract_body_onsets(mode, markerA_id, savepath, h_thres = 0.1,
                               vel_mode= "on" if metric == "vel" else "off")


### Faster Version

In [3]:
from itertools import product
from concurrent.futures import ProcessPoolExecutor

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": [9, 10], "a": 60, "b": 140, "metric": ["pos", "vel"]}
config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"],
           "markerA_id": [15, 16], "a": 60, "b": 140, "metric": ["pos", "vel"]}
configs = [config1, config2]

# Build a list of all tasks
tasks = []
for cfg in configs:
    for sub_dir, mode, markerA_id, metric in product(
        cfg["sub_dir"], cfg["mode"], cfg["markerA_id"], cfg["metric"]
    ):
        savepath = f"./extracted_body_onsets/{metric}"
        vel_mode = "on" if metric == "vel" else "off"
        tasks.append((mode, markerA_id, savepath, vel_mode))

# Worker wrapper
def _run_extraction(args):
    mode, markerA_id, savepath, vel_mode = args
    extract_body_onsets(mode, markerA_id, savepath, h_thres=0.1, vel_mode=vel_mode)

# Parallel execution
with ProcessPoolExecutor() as pool:
    pool.map(_run_extraction, tasks)


100%|██████████| 1510/1510 [01:08<00:00, 22.13it/s]



100%|██████████| 1510/1510 [01:08<00:00, 22.12it/s]


100%|██████████| 1510/1510 [01:08<00:00, 22.11it/s]

100%|██████████| 1510/1510 [01:08<00:00, 22.11it/s]


100%|██████████| 1510/1510 [01:08<00:00, 22.11it/s]


In [ ]:
fps = 60
a = 70; b =140

metric = "pos"
mode = "zero_uni"


onset_dir = f"./extracted_body_onsets/{metric}"
save_dir = f"./extracted_body_onsets/{metric}/combination"
f_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)


for idx, filename in enumerate(tqdm(aist_filelist)):

    
    test_path = os.path.join(onset_dir, "ax0", f"left_wrist_{mode}_{filename}")
    isExist = os.path.exists(test_path) 
    if not isExist:
        continue
                            
    left_hand_x  = load_pickle(os.path.join(onset_dir, "ax0", f"left_wrist_{mode}_{filename}"))
    left_hand_y  = load_pickle(os.path.join(onset_dir, "ax1", f"left_wrist_{mode}_{filename}"))
    
    right_hand_x = load_pickle(os.path.join(onset_dir, "ax0", f"right_wrist_{mode}_{filename}"))
    right_hand_y = load_pickle(os.path.join(onset_dir, "ax1", f"right_wrist_{mode}_{filename}"))
    
    left_foot_x  = load_pickle(os.path.join(onset_dir, "ax0", f"left_ankle_{mode}_{filename}"))
    left_foot_y  = load_pickle(os.path.join(onset_dir, "ax1", f"left_ankle_{mode}_{filename}"))
    
    right_foot_x = load_pickle(os.path.join(onset_dir, "ax0", f"right_ankle_{mode}_{filename}"))
    right_foot_y = load_pickle(os.path.join(onset_dir, "ax1", f"right_ankle_{mode}_{filename}"))
    
    novelty_length = left_hand_x['raw_signal'].shape[0]
    
    key = 'sensor_onsets'
    thres = 0.25
    
    bothhand_x = filter_dir_onsets_by_threshold((left_hand_x[key] + right_hand_x[key]), threshold_s= thres, fps=fps)
    bothhand_y = filter_dir_onsets_by_threshold((left_hand_y[key] + right_hand_y[key]), threshold_s= thres, fps=fps)

    bothfoot_x = filter_dir_onsets_by_threshold((left_foot_x[key] + right_foot_x[key]), threshold_s= thres, fps=fps)
    bothfoot_y = filter_dir_onsets_by_threshold((left_foot_y[key] + right_foot_y[key]), threshold_s= thres, fps=fps)
    
    lefthand_xy = filter_dir_onsets_by_threshold((left_hand_x[key] + left_hand_y[key]), threshold_s= thres, fps=fps)
    righthand_xy = filter_dir_onsets_by_threshold((right_hand_x[key] + right_hand_y[key]), threshold_s= thres, fps=fps)

    leftfoot_xy = filter_dir_onsets_by_threshold((left_foot_x[key] + left_foot_y[key]), threshold_s= thres, fps=fps)
    rightfoot_xy = filter_dir_onsets_by_threshold((right_foot_x[key] + right_foot_y[key]), threshold_s= thres, fps=fps)
    
    # Resultant part
    key1 = 'resultant_onsets'
    left_hand_resultant  = load_pickle(os.path.join(onset_dir, "resultant", f"left_wrist_{mode}_{filename}"))
    right_hand_resultant  = load_pickle(os.path.join(onset_dir, "resultant", f"right_wrist_{mode}_{filename}"))

    left_foot_resultant = load_pickle(os.path.join(onset_dir, "resultant", f"left_ankle_{mode}_{filename}"))
    right_foot_resultant = load_pickle(os.path.join(onset_dir, "resultant", f"right_ankle_{mode}_{filename}"))
    
    both_hand_resultant = filter_dir_onsets_by_threshold((left_hand_resultant[key1] + right_hand_resultant[key1]), threshold_s= thres, fps=fps)
    both_foot_resultant = filter_dir_onsets_by_threshold((left_foot_resultant[key1] + right_foot_resultant[key1]), threshold_s= thres, fps=fps)
    
    
    json_combi = {
        "bothhand_x": bothhand_x,
        "bothhand_y": bothhand_y,
        "lefthand_xy": lefthand_xy,
        "righthand_xy": righthand_xy,
        "bothfoot_x": bothfoot_x,
        "bothfoot_y": bothfoot_y,
        "leftfoot_xy": leftfoot_xy,
        "rightfoot_xy": rightfoot_xy,
        
        "both_hand_resultant": both_hand_resultant,
        "both_foot_resultant": both_foot_resultant
    }
    
    
    sv_fname = f"{filename}_{mode}_combi_onsets.pkl"
    fpath2 = os.path.join(save_dir, sv_fname)
    save_to_pickle(fpath2, json_combi)
    
    

## Estimate Tempo - Discrete

In [5]:
json_filename = "music_id_tempo.json"
with open(json_filename, "r") as file:
    aist_tempo = json.load(file)
    
def create_dir(main_dir, tempo_dir):
    # main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
    directories = [f"{tempo_dir}/pos", f"{tempo_dir}/vel",
                   f"{tempo_dir}/tempo_data/pos", f"{tempo_dir}/tempo_data/vel",]
    
    for dir_path in directories:
        full_path = os.path.join(main_dir, dir_path)
        os.makedirs(full_path, exist_ok=True)
        
segment_keys = ["both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y", 
                "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy", 
                "left_hand_x", "right_hand_x", "left_hand_y", "right_hand_y", 
                "left_foot_x", "right_foot_x", "left_foot_y", "right_foot_y", 
                "both_hand_resultant", "both_foot_resultant", "left_hand_resultant", 
                "right_hand_resultant", "left_foot_resultant", "right_foot_resultant"]

result = { key: {
    "filename": [],
    "dance_genre": [],
    "situation": [],
    "camera_id": [],
    "dancer_id": [],
    "music_id": [],
    "choreo_id": [],
    "music_tempo": [],
    "estimated_bpm_per_window": [],
    "magnitude_per_window": [],
    "bpm_avg": [],
    "bpm_mode": [],
    "bpm_median": [],
} for key in segment_keys }

fps = 60
w_sec = 5
h_sec = w_sec/2
window_size = int(fps*w_sec)
hop_size = int(fps*h_sec)

a = 30; b =140
tempi_range = np.arange(a,b,1)
metric = "vel"
mode = "zero_uni"

main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
create_dir(main_dir, f"tempo_{a}_{b}")

save_dir = f"./saved_result/tempo_{a}_{b}/"
onset_dir = f"./extracted_body_onsets/{metric}/"
f_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)


count= 0
for idx, filename in enumerate(tqdm(aist_filelist)):
    
    file_info = filename.split("_")
    dance_genre = file_info[0] 
    situation = file_info[1] 
    camera_id = file_info[2] 
    dancer_id = file_info[3]
    music_id = file_info[4]
    choreo_id = file_info[5].strip(".pkl")
    
    test_path = os.path.join(onset_dir, "ax0", f"left_wrist_{mode}_{filename}")
    isExist = os.path.exists(test_path) 
    if not isExist:
        continue
                            
    left_hand_x  = load_pickle(os.path.join(onset_dir, "ax0", f"left_wrist_{mode}_{filename}"))
    left_hand_y  = load_pickle(os.path.join(onset_dir, "ax1", f"left_wrist_{mode}_{filename}"))
    
    right_hand_x = load_pickle(os.path.join(onset_dir, "ax0", f"right_wrist_{mode}_{filename}"))
    right_hand_y = load_pickle(os.path.join(onset_dir, "ax1", f"right_wrist_{mode}_{filename}"))
    
    left_foot_x  = load_pickle(os.path.join(onset_dir, "ax0", f"left_ankle_{mode}_{filename}"))
    left_foot_y  = load_pickle(os.path.join(onset_dir, "ax1", f"left_ankle_{mode}_{filename}"))
    
    right_foot_x = load_pickle(os.path.join(onset_dir, "ax0", f"right_ankle_{mode}_{filename}"))
    right_foot_y = load_pickle(os.path.join(onset_dir, "ax1", f"right_ankle_{mode}_{filename}"))
    
    novelty_length = left_hand_x['raw_signal'].shape[0]
    
    key = 'sensor_onsets'       #   sensor_abs_pos_filtered
    thres = 0.3     # time threshold
    
    bothhand_x = filter_dir_onsets_by_threshold((left_hand_x[key] + right_hand_x[key]), threshold_s= thres, fps=fps)
    bothhand_y = filter_dir_onsets_by_threshold((left_hand_y[key] + right_hand_y[key]), threshold_s= thres, fps=fps)

    bothfoot_x = filter_dir_onsets_by_threshold((left_foot_x[key] + right_foot_x[key]), threshold_s= thres, fps=fps)
    bothfoot_y = filter_dir_onsets_by_threshold((left_foot_y[key] + right_foot_y[key]), threshold_s= thres, fps=fps)
    
    lefthand_xy = filter_dir_onsets_by_threshold((left_hand_x[key] + left_hand_y[key]), threshold_s= thres, fps=fps)
    righthand_xy = filter_dir_onsets_by_threshold((right_hand_x[key] + right_hand_y[key]), threshold_s= thres, fps=fps)

    leftfoot_xy = filter_dir_onsets_by_threshold((left_foot_x[key] + left_foot_y[key]), threshold_s= thres, fps=fps)
    rightfoot_xy = filter_dir_onsets_by_threshold((right_foot_x[key] + right_foot_y[key]), threshold_s= thres, fps=fps)
    
    # Resultant part
    key1 = 'resultant_onsets'
    left_hand_resultant  = load_pickle(os.path.join(onset_dir, "resultant", f"left_wrist_{mode}_{filename}"))
    right_hand_resultant  = load_pickle(os.path.join(onset_dir, "resultant", f"right_wrist_{mode}_{filename}"))

    left_foot_resultant = load_pickle(os.path.join(onset_dir, "resultant", f"left_ankle_{mode}_{filename}"))
    right_foot_resultant = load_pickle(os.path.join(onset_dir, "resultant", f"right_ankle_{mode}_{filename}"))
    
    both_hand_resultant = filter_dir_onsets_by_threshold((left_hand_resultant[key1] + right_hand_resultant[key1]), threshold_s= thres, fps=fps)
    both_foot_resultant = filter_dir_onsets_by_threshold((left_foot_resultant[key1] + right_foot_resultant[key1]), threshold_s= thres, fps=fps)
    
    segment_ax = {
                "both_hand_x": bothhand_x, "both_hand_y": bothhand_y, "both_foot_x": bothfoot_x, "both_foot_y": bothfoot_y,
                "lefthand_xy": lefthand_xy, "righthand_xy": righthand_xy, "leftfoot_xy": leftfoot_xy, "rightfoot_xy": rightfoot_xy,
                
                "left_hand_x": left_hand_x[key], "right_hand_x": right_hand_x[key], 
                "left_hand_y": left_hand_y[key], "right_hand_y": right_hand_y[key],
                
                "left_foot_x": left_foot_x[key], "right_foot_x": right_foot_x[key],
                "left_foot_y": left_foot_y[key], "right_foot_y": right_foot_y[key],
                
                "both_hand_resultant": both_hand_resultant, "both_foot_resultant": both_foot_resultant,                         
                "left_hand_resultant": left_hand_resultant[key1], "right_hand_resultant": right_hand_resultant[key1],
                "left_foot_resultant": left_foot_resultant[key1], "right_foot_resultant": right_foot_resultant[key1],
                }
    tempo_data = {}
    for seg_key, seg in segment_ax.items():
        
        sensor_onsets = binary_to_peak(seg, peak_duration=0.05)
        
        tempogram_ab, tempogram_raw, time_axis_seconds, tempo_axis_bpm = compute_tempogram(sensor_onsets, fps, 
                                                                        window_length=window_size, hop_size=hop_size, tempi=tempi_range)
        

        tempo_data_maxmethod = dance_beat_tempo_estimation_maxmethod(tempogram_ab, tempogram_raw, fps, 
                                                        novelty_length, window_size, hop_size, tempi_range)
    
        tempo_data[seg_key] = tempo_data_maxmethod
        
        estimated_bpm_per_window = tempo_data_maxmethod["bpm_arr"]
        magnitude_per_window = tempo_data_maxmethod["mag_arr"]
        
        tempo_avg = np.round(np.average(estimated_bpm_per_window), 2)     # mean
        tempo_mode = stats.mode(estimated_bpm_per_window.flatten())[0]        # 
        tempo_median = np.median(estimated_bpm_per_window.flatten())

        # Append the rows to the DataFrame
        result[seg_key]["filename"].append(filename.strip(".pkl"))
        result[seg_key]["dance_genre"].append(dance_genre)
        result[seg_key]["situation"].append(situation)
        result[seg_key]["camera_id"].append(camera_id)
        result[seg_key]["dancer_id"].append(dancer_id)
        result[seg_key]["music_id"].append(music_id)
        result[seg_key]["choreo_id"].append(choreo_id)
        result[seg_key]["music_tempo"].append(aist_tempo[music_id])
        result[seg_key]["estimated_bpm_per_window"].append(estimated_bpm_per_window)
        result[seg_key]["magnitude_per_window"].append(magnitude_per_window)
        result[seg_key]["bpm_avg"].append(tempo_avg)
        result[seg_key]["bpm_mode"].append(tempo_mode)
        result[seg_key]["bpm_median"].append(tempo_median)
    
    # tempodata_fname = f"tempo_data/{metric}/{filename[:-4]}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}_tempo_data.pkl"
    # fpath2 = os.path.join(save_dir, tempodata_fname)
    # save_to_pickle(fpath2, tempo_data)
    
    count +=1
print("total processed:",count)    
for seg_key in segment_keys:
    
    fname1 = f"{metric}/{seg_key}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
    fpath1 = os.path.join(save_dir, fname1)
    df_seg = pd.DataFrame(result[seg_key])
    df_seg.to_pickle(fpath1)
    
#     # tempodata_fname = f"tempo_data/{metric}/{seg_key}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}_tempo_data.pkl"
#     # fpath2 = os.path.join(save_dir, tempodata_fname)
#     # save_to_pickle(fpath2, tempo_data[seg_key])    
#     print(f"Saved {fname1}")

100%|██████████| 1510/1510 [09:27<00:00,  2.66it/s]


total processed: 1341


In [ ]:
def min_max_normalize(data):
    min_vals = np.min(data)  # Minimum values along each column
    max_vals = np.max(data)  # Maximum values along each column
    normalized_data = (data - min_vals) / (max_vals - min_vals)
    return normalized_data

#### Estimate Tempo - Tempogram combination method

In [16]:
json_filename = "music_id_tempo.json"
with open(json_filename, "r") as file:
    aist_tempo = json.load(file)
    
def create_dir(main_dir, tempo_dir):
    # main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
    directories = [f"{tempo_dir}/pos", f"{tempo_dir}/vel",
                   f"{tempo_dir}/tempo_data/pos", f"{tempo_dir}/tempo_data/vel",]
    
    for dir_path in directories:
        full_path = os.path.join(main_dir, dir_path)
        os.makedirs(full_path, exist_ok=True)
        
segment_keys = ["both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y", 
                "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy", 
                "left_hand_x", "right_hand_x", "left_hand_y", "right_hand_y", 
                "left_foot_x", "right_foot_x", "left_foot_y", "right_foot_y", 
                "both_hand_resultant", "both_foot_resultant", "left_hand_resultant", 
                "right_hand_resultant", "left_foot_resultant", "right_foot_resultant"]

result ={
    "filename": [],
    "dance_genre": [],
    "situation": [],
    "camera_id": [],
    "dancer_id": [],
    "music_id": [],
    "choreo_id": [],
    "music_tempo": [],
    "estimated_bpm_per_window": [],
    "magnitude_per_window": [],
    "bpm_avg": [],
    "bpm_mode": [],
    "bpm_median": [],
}

fps = 60
w_sec = 5
h_sec = w_sec/2
window_size = int(fps*w_sec)
hop_size = int(fps*h_sec)

a = 60; b =140
tempi_range = np.arange(a,b,1)
metric = "pos"
mode = "zero_uni"

main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result"
create_dir(main_dir, f"tttempo_{a}_{b}")

save_dir = f"./saved_result/tttempo_{a}_{b}/"
f_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)

pos_onset_dir = f"./extracted_body_onsets/pos/"
vel_onset_dir = f"./extracted_body_onsets/vel/"


count= 0
for idx, filename in enumerate(tqdm(aist_filelist)):
    
    file_info = filename.split("_")
    dance_genre = file_info[0] 
    situation = file_info[1] 
    camera_id = file_info[2] 
    dancer_id = file_info[3]
    music_id = file_info[4]
    choreo_id = file_info[5].strip(".pkl")
    
    test_path = os.path.join(pos_onset_dir, "ax0", f"left_wrist_{mode}_{filename}")
    isExist = os.path.exists(test_path) 
    if not isExist:
        continue
    
    left_hand_x  = load_pickle(os.path.join(pos_onset_dir, "ax0", f"left_wrist_{mode}_{filename}"))
    left_hand_y  = load_pickle(os.path.join(pos_onset_dir, "ax1", f"left_wrist_{mode}_{filename}"))

    right_hand_x = load_pickle(os.path.join(pos_onset_dir, "ax0", f"right_wrist_{mode}_{filename}"))
    right_hand_y = load_pickle(os.path.join(pos_onset_dir, "ax1", f"right_wrist_{mode}_{filename}"))

    left_foot_x  = load_pickle(os.path.join(pos_onset_dir, "ax0", f"left_ankle_{mode}_{filename}"))
    left_foot_y  = load_pickle(os.path.join(pos_onset_dir, "ax1", f"left_ankle_{mode}_{filename}"))

    right_foot_x = load_pickle(os.path.join(pos_onset_dir, "ax0", f"right_ankle_{mode}_{filename}"))
    right_foot_y = load_pickle(os.path.join(pos_onset_dir, "ax1", f"right_ankle_{mode}_{filename}"))

    novelty_length = left_hand_x['raw_signal'].shape[0]

    #### Velocity part ####

    v_left_hand_x  = load_pickle(os.path.join(vel_onset_dir, "ax0", f"left_wrist_{mode}_{filename}"))
    v_left_hand_y  = load_pickle(os.path.join(vel_onset_dir, "ax1", f"left_wrist_{mode}_{filename}"))

    v_right_hand_x = load_pickle(os.path.join(vel_onset_dir, "ax0", f"right_wrist_{mode}_{filename}"))
    v_right_hand_y = load_pickle(os.path.join(vel_onset_dir, "ax1", f"right_wrist_{mode}_{filename}"))

    v_left_foot_x  = load_pickle(os.path.join(vel_onset_dir, "ax0", f"left_ankle_{mode}_{filename}"))
    v_left_foot_y  = load_pickle(os.path.join(vel_onset_dir, "ax1", f"left_ankle_{mode}_{filename}"))

    v_right_foot_x = load_pickle(os.path.join(vel_onset_dir, "ax0", f"right_ankle_{mode}_{filename}"))
    v_right_foot_y = load_pickle(os.path.join(vel_onset_dir, "ax1", f"right_ankle_{mode}_{filename}"))

    #### Filtering Onsets ####

    key = 'sensor_onsets'  # or 'sensor_abs_vel_filtered', depending on your data
    thres = 0.3            # time threshold

    # Position-based filtered onsets
    bothhand_x = filter_dir_onsets_by_threshold((left_hand_x[key] + right_hand_x[key]), threshold_s=thres, fps=fps)
    bothhand_y = filter_dir_onsets_by_threshold((left_hand_y[key] + right_hand_y[key]), threshold_s=thres, fps=fps)

    bothfoot_x = filter_dir_onsets_by_threshold((left_foot_x[key] + right_foot_x[key]), threshold_s=thres, fps=fps)
    bothfoot_y = filter_dir_onsets_by_threshold((left_foot_y[key] + right_foot_y[key]), threshold_s=thres, fps=fps)

    lefthand_xy = filter_dir_onsets_by_threshold((left_hand_x[key] + left_hand_y[key]), threshold_s=thres, fps=fps)
    righthand_xy = filter_dir_onsets_by_threshold((right_hand_x[key] + right_hand_y[key]), threshold_s=thres, fps=fps)

    leftfoot_xy = filter_dir_onsets_by_threshold((left_foot_x[key] + left_foot_y[key]), threshold_s=thres, fps=fps)
    rightfoot_xy = filter_dir_onsets_by_threshold((right_foot_x[key] + right_foot_y[key]), threshold_s=thres, fps=fps)

    # Velocity-based filtered onsets
    v_bothhand_x = filter_dir_onsets_by_threshold((v_left_hand_x[key] + v_right_hand_x[key]), threshold_s=thres, fps=fps)
    v_bothhand_y = filter_dir_onsets_by_threshold((v_left_hand_y[key] + v_right_hand_y[key]), threshold_s=thres, fps=fps)

    v_bothfoot_x = filter_dir_onsets_by_threshold((v_left_foot_x[key] + v_right_foot_x[key]), threshold_s=thres, fps=fps)
    v_bothfoot_y = filter_dir_onsets_by_threshold((v_left_foot_y[key] + v_right_foot_y[key]), threshold_s=thres, fps=fps)

    v_lefthand_xy = filter_dir_onsets_by_threshold((v_left_hand_x[key] + v_left_hand_y[key]), threshold_s=thres, fps=fps)
    v_righthand_xy = filter_dir_onsets_by_threshold((v_right_hand_x[key] + v_right_hand_y[key]), threshold_s=thres, fps=fps)

    v_leftfoot_xy = filter_dir_onsets_by_threshold((v_left_foot_x[key] + v_left_foot_y[key]), threshold_s=thres, fps=fps)
    v_rightfoot_xy = filter_dir_onsets_by_threshold((v_right_foot_x[key] + v_right_foot_y[key]), threshold_s=thres, fps=fps)
    

    ###########################################################################################    
    sensor_onsets1 = binary_to_peak(bothhand_y, peak_duration=0.05)
    sensor_onsets2 = binary_to_peak(bothfoot_y, peak_duration=0.05)
    
    sensor_onsets3 = binary_to_peak(v_bothhand_y, peak_duration=0.05)   # v_bothhand_y = np.diff(bothhand_y)
    sensor_onsets4 = binary_to_peak(v_bothfoot_y, peak_duration=0.05)
    sensor_onsets3 = np.append(sensor_onsets3, 0).reshape(-1, 1)
    sensor_onsets4 = np.append(sensor_onsets4, 0).reshape(-1, 1)
    
    # Compute tempograms for each sensor onset sequence
    tempogram_ab1, tempogram_raw1, _, _ = compute_tempogram(sensor_onsets1, fps, 
                                        window_length=window_size, hop_size=hop_size, tempi=tempi_range)

    tempogram_ab2, tempogram_raw2, _, _ = compute_tempogram(sensor_onsets2, fps, 
                                        window_length=window_size, hop_size=hop_size, tempi=tempi_range)

    tempogram_ab3, tempogram_raw3, _, _ = compute_tempogram(sensor_onsets3, fps, 
                                        window_length=window_size, hop_size=hop_size, tempi=tempi_range)

    tempogram_ab4, tempogram_raw4, _, _ = compute_tempogram(sensor_onsets4, fps, 
                                        window_length=window_size, hop_size=hop_size, tempi=tempi_range)

    # getting accuracy 60.62%
    # tempogram_ab = np.mean([tempogram_ab1, tempogram_ab2, tempogram_ab3, tempogram_ab4], axis=0)
    # tempogram_raw = np.mean([tempogram_raw1, tempogram_raw2, tempogram_raw3, tempogram_raw4], axis=0)
    
    # Compute adaptive weights based on peak magnitudes of tempograms
    def adaptive_weights(tempograms):
        peak_magnitudes = np.array([np.max(t) for t in tempograms])
        return peak_magnitudes / np.sum(peak_magnitudes)

    # Tempogram lists
    tempograms_ab = [tempogram_ab1[0], tempogram_ab2[0], tempogram_ab3[0], tempogram_ab4[0]]
    tempograms_raw = [tempogram_raw1[0], tempogram_raw2[0], tempogram_raw3[0], tempogram_raw4[0]]

    # Compute weights
    weights_ab = adaptive_weights(tempograms_ab)
    weights_raw = adaptive_weights(tempograms_raw)

    # Compute weighted average of tempograms

    # Weighted tempogram_ab
    tempogram_ab = [sum(w * t for w, t in zip(weights_ab, tempograms_ab))]

    # Weighted tempogram_raw
    tempogram_raw = [sum(w * t for w, t in zip(weights_raw, tempograms_raw))]
    
    

    tempo_data_maxmethod = dance_beat_tempo_estimation_maxmethod(tempogram_ab, tempogram_raw, fps, 
                                                    novelty_length, window_size, hop_size, tempi_range)
    
    
    
    
    #############################################################################################
    estimated_bpm_per_window = tempo_data_maxmethod["bpm_arr"]
    magnitude_per_window = tempo_data_maxmethod["mag_arr"]
    
    tempo_avg = np.round(np.average(estimated_bpm_per_window), 2)     # mean
    tempo_mode = stats.mode(estimated_bpm_per_window.flatten())[0]        # 
    tempo_median = np.median(estimated_bpm_per_window.flatten())

    # Append the rows to the DataFrame
    result["filename"].append(filename.strip(".pkl"))
    result["dance_genre"].append(dance_genre)
    result["situation"].append(situation)
    result["camera_id"].append(camera_id)
    result["dancer_id"].append(dancer_id)
    result["music_id"].append(music_id)
    result["choreo_id"].append(choreo_id)
    result["music_tempo"].append(aist_tempo[music_id])
    result["estimated_bpm_per_window"].append(estimated_bpm_per_window)
    result["magnitude_per_window"].append(magnitude_per_window)
    result["bpm_avg"].append(tempo_avg)
    result["bpm_mode"].append(tempo_mode)
    result["bpm_median"].append(tempo_median)
    
    # tempodata_fname = f"tempo_data/{metric}/{filename[:-4]}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}_tempo_data.pkl"
    # fpath2 = os.path.join(save_dir, tempodata_fname)
    # save_to_pickle(fpath2, tempo_data)
    
fname1 = f"{metric}/bhf_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
fpath1 = os.path.join(save_dir, fname1)
df_seg = pd.DataFrame(result)
df_seg.to_pickle(fpath1)
    
#     # tempodata_fname = f"tempo_data/{metric}/{seg_key}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}_tempo_data.pkl"
#     # fpath2 = os.path.join(save_dir, tempodata_fname)
#     # save_to_pickle(fpath2, tempo_data[seg_key])    
#     print(f"Saved {fname1}")

100%|██████████| 1510/1510 [01:02<00:00, 24.02it/s]


In [13]:
tempogram_raw.shape

(80, 4)

In [ ]:
####### method codes ##########

# getting accuracy 60.62%
    # tempogram_ab = np.mean([tempogram_ab1, tempogram_ab2, tempogram_ab3, tempogram_ab4], axis=0)
    # tempogram_raw = np.mean([tempogram_raw1, tempogram_raw2, tempogram_raw3, tempogram_raw4], axis=0)
    
    # Compute adaptive weights based on peak magnitudes of tempograms
    # def adaptive_weights(tempograms):
    #     peak_magnitudes = np.array([np.max(t) for t in tempograms])
    #     return peak_magnitudes / np.sum(peak_magnitudes)

    # # Tempogram lists
    # tempograms_ab = [tempogram_ab1[0], tempogram_ab2[0], tempogram_ab3[0], tempogram_ab4[0]]
    # tempograms_raw = [tempogram_raw1[0], tempogram_raw2[0], tempogram_raw3[0], tempogram_raw4[0]]

    # # Compute weights
    # weights_ab = adaptive_weights(tempograms_ab)
    # weights_raw = adaptive_weights(tempograms_raw)

    # # Compute weighted average of tempograms

    # # Weighted tempogram_ab
    # tempogram_ab = [sum(w * t for w, t in zip(weights_ab, tempograms_ab))]

    # # Weighted tempogram_raw
    # tempogram_raw = [sum(w * t for w, t in zip(weights_raw, tempograms_raw))]
    
    # -----------------------------------------------
    # getting accuracy 58.24%
    # tempogram_ab = np.max([tempogram_ab1, tempogram_ab2], axis=0)
    # tempogram_raw = np.max([tempogram_raw1, tempogram_raw2], axis=0)
    
    # getting accuracy 54.43%
    # tempogram_ab = np.max([tempogram_ab3, tempogram_ab4], axis=0)
    # tempogram_raw = np.max([tempogram_raw3, tempogram_raw4], axis=0)
    
    # # Stability part
    # tempogram_stack = np.stack([tempogram_ab1, tempogram_ab2, tempogram_ab3, tempogram_ab4], axis=-1)
    # stability = np.std(tempogram_stack, axis=-1)
    # tempogram_ab = np.mean(tempogram_stack, axis=-1) / (1 + stability)

    # # Stability part
    # tempogram_raw_stack = np.stack([tempogram_raw1, tempogram_raw2, tempogram_raw3, tempogram_raw4], axis=-1)
    # stability_raw = np.std(tempogram_raw_stack, axis=-1)
    # tempogram_raw = np.mean(tempogram_raw_stack, axis=-1) / (1 + stability_raw)